In [1]:
import numpy as np
import pandas as pd

# LifeTabel

In [2]:
LifeTabel=pd.read_csv("LifeTabel100.csv")
LifeTabel.head()

,Age,l_x,d_x,qx,px
0,0,100000,1734,0.017340,0.982660
1,1,98266,122,0.001242,0.998758
2,2,98144,92,0.000937,0.999063
3,3,98052,78,0.000795,0.999205
4,4,97974,71,0.000725,0.999275


# LifeTabel monthly

In [3]:
Year=np.repeat(range(0,100) ,12)
Year1=Year.tolist()
month=np.tile([1,2,3,4,5,6,7,8,9,10,11,12], 100)
month1=month.tolist()
Age=[]
for i in range(0,1200):
    Age.append(Year1[i]+month1[i]/12)
    
    
L_x=[]
for i in range(0,100):
    for j in range(1,13):
        L_x.append( LifeTabel.l_x[i]-LifeTabel.d_x[i]*(j/12))
        
        
d_x=[]
for i in range(0,1199):
    d_x.append(   L_x[i]-L_x[i+1]) 
d_x.append(671)


q_x=[]
p_x=[]
for i in range(0,1200):
    q_x.append(d_x[i]/ L_x[i])
    p_x.append(1-q_x[i])

In [4]:
LifeTabel_month=pd.DataFrame()
LifeTabel_month["Age"]=Age
LifeTabel_month["year"]=Year1
LifeTabel_month["month"]=month.tolist()
LifeTabel_month["L_x"]=L_x
LifeTabel_month["d_x"]=d_x
LifeTabel_month["q_x"]=q_x
LifeTabel_month["p_x"]=p_x
LifeTabel_month.head()

,Age,year,month,L_x,d_x,q_x,p_x
0,0.083333,0,1,99855.5,144.5,0.001447,0.998553
1,0.166667,0,2,99711.0,144.5,0.001449,0.998551
2,0.250000,0,3,99566.5,144.5,0.001451,0.998549
3,0.333333,0,4,99422.0,144.5,0.001453,0.998547
4,0.416667,0,5,99277.5,144.5,0.001456,0.998544


# Interest Tabel Month

In [5]:
InterestTabel_month=pd.DataFrame()
InterestTabel_month["Interest"]=np.repeat((12*((1+0.15)**(1/12)-1)) ,62)
InterestTabel_month["nu"]=np.repeat(1/(1+InterestTabel_month.Interest[0]) ,62)
nu_n=[]
for i in range(0,62): 
    nu_n.append( InterestTabel_month.nu[i]**(i/12))

InterestTabel_month["nu_n"]=nu_n
InterestTabel_month=InterestTabel_month[0:61]

InterestTabel_month.head()

,Interest,nu,nu_n
0,0.140579,0.876748,1.000000
1,0.140579,0.876748,0.989099
2,0.140579,0.876748,0.978316
3,0.140579,0.876748,0.967651
4,0.140579,0.876748,0.957102


# Interest_Tabe

In [6]:
Interest_Tabe=pd.DataFrame()

# insert
x = np.array([0.15, 0.10])
Interest=np.repeat(x, [11, 20], axis=0)

#Nu and d
Nu=[]
Nu_n=[]
d=[]
for i in range(0,31):
    Nu.append(1/(1+Interest[i]))
    Nu_n.append(Nu[i]**i)
    d.append(Interest[i]/(1+Interest[i]))
    
Interest_Tabe["Interest"]=Interest
Interest_Tabe["Nu"]=Nu
Interest_Tabe["Nu_n"]=Nu_n
Interest_Tabe["d"]=d

Interest_Tabe.head()

,Interest,Nu,Nu_n,d
0,0.15,0.869565,1.000000,0.130435
1,0.15,0.869565,0.869565,0.130435
2,0.15,0.869565,0.756144,0.130435
3,0.15,0.869565,0.657516,0.130435
4,0.15,0.869565,0.571753,0.130435


# Term insurance annual
# محصول عمر 

In [7]:
def Term_insurance_annual(Age=None , period=None , suminsuraned=None , add=False):
    global  EPV_TIA , Cost,Wage,data_TIA
    epv_TIA=[]
    data_TIA=pd.DataFrame()
    data_TIA['Age']=range(Age,Age+1)
    for j in range(0,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append (Interest_Tabe.Nu_n[j+1]*((LifeTabel.l_x[age+j]-LifeTabel.l_x[age+j+1])/LifeTabel.l_x[age]))
        data_TIA[j]=A_xn
    for N  in range(0,period):
        epv_TIA.append( data_TIA[:][N])

    EPV= (sum(epv_TIA).values)*suminsuraned
    Cost=(0.02*EPV[0])+(0.003*suminsuraned)
    Wage=(0.05*EPV[0])
    EPV_TIA=EPV[0]+Cost+Wage
    if(add==False):
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              سرمایه فوت : ", f'{suminsuraned:,}')
        print("   حق بیمه خالص : "  ,  f'{round(EPV[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_TIA,2):,}' ," تومان")
        
    if(add==True):
        return EPV_TIA

In [9]:
Term_insurance_annual(Age=20,period=20,suminsuraned=1000000000 ,add=False )

سن :  20          مدت قرارداد :  20               سرمایه فوت :  1,000,000,000
   حق بیمه خالص :  8,823,845.68  تومان
   کارمزد و هزینه اداری :  3,617,669.2  تومان
   حق بیمه نهایی :  12,441,514.88  تومان


# Pure endowment
# محصول پس انداز

In [10]:
def Pure_endowment(Age=None , period=None , suminsuraned=1000000000 , add=False):
    


    global  epv_PE , Cost,Wage
    epv_PE=(LifeTabel.l_x[Age+period]/LifeTabel.l_x[Age])*Interest_Tabe.Nu_n[period]
    EPV= epv_PE*suminsuraned
    Cost=(0.02*EPV)+(0.003*suminsuraned)
    Wage=(0.05*EPV)
    epv_PE=EPV+Cost+Wage
    if(add==False):
        
         print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              سرمایه پس انداز : ", f'{suminsuraned:,}')
         print("   حق بیمه خالص : "  ,  f'{round(EPV,2):,}' ," تومان")
         print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
         print("   حق بیمه نهایی : "  ,  f'{round(epv_PE,2):,}' ," تومان")
        
        
    if(add==True):
        return epv_PE

In [11]:
Pure_endowment(Age=20,period= 20,suminsuraned=1000000000 )

سن :  20          مدت قرارداد :  20               سرمایه پس انداز :  1,000,000,000
   حق بیمه خالص :  144,797,942.79  تومان
   کارمزد و هزینه اداری :  13,135,856.0  تومان
   حق بیمه نهایی :  157,933,798.78  تومان


# Endowment
# محصول عمر و پس انداز

In [12]:
def endowment(Age=None   , period=None   , death_capital=None   , Saving_capital=None , add=False):
    global  EPV_pe , Cost,Wage , EPV_E
    global  EPV_TIA , Cost,Wage
    epv_PE=(LifeTabel.l_x[Age+period]/LifeTabel.l_x[Age])*Interest_Tabe.Nu_n[period]
    EPV_pe= epv_PE*Saving_capital

    
    epv_TIA=[]
    data_TIA=pd.DataFrame()
    data_TIA['Age']=range(Age,Age+1)
    for j in range(0,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append (Interest_Tabe.Nu_n[j+1]*((LifeTabel.l_x[age+j]-LifeTabel.l_x[age+j+1])/LifeTabel.l_x[age]))
        data_TIA[j]=A_xn
    for N  in range(0,period):
        epv_TIA.append( data_TIA[:][N])
    EPV_TIA= (sum(epv_TIA).values)*death_capital

    EPV_E= EPV_TIA+EPV_pe
    Cost=((0.02*EPV_E)+(0.003*death_capital))
    if(add==False):    
        
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              سرمایه فوت : ", f'{death_capital:,}' ,  "              سرمایه پس انداز : ", f'{Saving_capital:,}')
        print("   حق بیمه خالص : "  ,  f'{round(EPV_E[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(((0.05*(EPV_E))+Cost)[0],2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_E[0]+((0.05*(EPV_E))+Cost)[0],2):,}' ," تومان")
        
        
    if(add==True):
        return EPV_E

    
    

In [15]:
endowment(Age=25,period=10,death_capital=1000000000,Saving_capital=1000000000)

سن :  25          مدت قرارداد :  10               سرمایه فوت :  1,000,000,000               سرمایه پس انداز :  1,000,000,000
   حق بیمه خالص :  250,333,607.11  تومان
   کارمزد و هزینه اداری :  20,523,352.5  تومان
   حق بیمه نهایی :  270,856,959.61  تومان


# Term annuity-due
# مستمری سر فصلی

In [16]:
def Term_annuity_due(Age=None  , period=None , pension_amount=None , add=False):
    global  data_ , Cost,Wage,EPV_
    epv_=[]
    data_=pd.DataFrame()
    data_['Age']=range(Age,Age+1)
    
    for j in range(0,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_[j]=A_xn

        
    for N  in range(0,period):
        epv_.append( data_[:][N])
    
    
    EPV= (sum(epv_).values)*pension_amount
    Cost=(0.02*EPV[0])
    Wage=(0.05*EPV[0])
    
    EPV_=EPV[0]+Cost+Wage
    if(add==False):      

        
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              مبلغ مستمری : ", f'{pension_amount:,}')
        print("   حق بیمه خالص : "  ,  f'{round(EPV[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_,2):,}' ," تومان")
           
    if(add==True):  
        return EPV_

In [18]:
Term_annuity_due(Age=25,period=20,pension_amount=5000000 , add=False)

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  5,000,000
   حق بیمه خالص :  40,836,876.17  تومان
   کارمزد و هزینه اداری :  2,858,581.33  تومان
   حق بیمه نهایی :  43,695,457.51  تومان


# Immediate life annuities
# مستمری ته فصلی

In [19]:
def Term_annuity_Immediate(Age=None  , period=None , pension_amount=None , add=False):
    global  data_I , Cost,Wage,EPV_I
    epv_I=[]
    data_I=pd.DataFrame()
    data_I['Age']=range(Age,Age+1)
    
    for j in range(1,period+1):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_I[j]=A_xn
        
    for N  in range(1,period+1):
        epv_I.append( data_I[:][N])
    
    
    EPV= (sum(epv_I).values)*pension_amount
    Cost=(0.02*EPV[0])
    Wage=(0.05*EPV[0])
    
    EPV_I=EPV[0]+Cost+Wage
    if(add==False):      
        
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              مبلغ مستمری : ", f'{pension_amount:,}')
        print("   حق بیمه خالص : "  ,  f'{round(EPV[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_I,2):,}' ," تومان")
        
    if(add==True):  
        return EPV_I

In [21]:
Term_annuity_Immediate(Age=25,period=10,pension_amount=5000000)

سن :  25          مدت قرارداد :  10               مبلغ مستمری :  5,000,000
   حق بیمه خالص :  24,957,134.88  تومان
   کارمزد و هزینه اداری :  1,746,999.44  تومان
   حق بیمه نهایی :  26,704,134.32  تومان


# Defer annuti
# مستمری تعویقی

In [22]:
def Defer_annuity_due(Age=None  , period=None , pension_amount=None , Defer=None , add=False):
    global  data_D, Cost,Wage,EPV_D
    epv_D=[]
    data_D=pd.DataFrame()
    data_D['Age']=range(Age,Age+1)
    
    for j in range(Defer,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_D[j]=A_xn
        
    for N  in range(Defer,period):
        epv_D.append( data_D[:][N])
    
    
    EPV= (sum(epv_D).values)*pension_amount
    Cost=(0.02*EPV[0])
    Wage=(0.05*EPV[0])
    
    EPV_D=EPV[0]+Cost+Wage
    if(add==False):      
        
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "                دوره تعویق  : ",Defer, "              مبلغ مستمری : ", f'{pension_amount:,}'    )
        print("   حق بیمه خالص : "  ,  f'{round(EPV[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_D,2):,}' ," تومان")
        
    if(add==True):  
        return EPV_D
        

In [24]:
Defer_annuity_due(Age=20,period=20,pension_amount=5000000,Defer=10)

سن :  20          مدت قرارداد :  20                 دوره تعویق  :  10               مبلغ مستمری :  5,000,000
   حق بیمه خالص :  12,131,881.44  تومان
   کارمزد و هزینه اداری :  849,231.7  تومان
   حق بیمه نهایی :  12,981,113.14  تومان


# Geometrically increasing annuit
# مستمری افزایشی

In [25]:
def Term_annuity_increasing(Age=None  , period=None , pension_amount=None , Rate_increase=None , add=False):
    global  data_increasing , Cost,Wage,EPV_increasing
    epv_increasing=[]
    data_increasing=pd.DataFrame()
    data_increasing['Age']=range(Age,Age+1)
    
    for j in range(0,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append (((1+Rate_increase)**j)*(Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_increasing[j]=A_xn

        
    for N  in range(0,period):
        epv_increasing.append( data_increasing[:][N])
    
    
    EPV= (sum(epv_increasing).values)*pension_amount
    Cost=(0.02*EPV[0])
    Wage=(0.05*EPV[0])
    
    EPV_increasing=EPV[0]+Cost+Wage
    if(add==False):  
        
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              مبلغ مستمری : ", f'{pension_amount:,}'     ,  "                   درصد افزایش مبلغ مستمری  :",Rate_increase)
        print("   حق بیمه خالص : "  ,  f'{round(EPV[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_increasing,2):,}' ," تومان")
    if(add==True):          
         return EPV_increasing

In [26]:
Term_annuity_increasing(Age=25,period=20,pension_amount=5000000,Rate_increase=0.1)

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  5,000,000                    درصد افزایش مبلغ مستمری  : 0.1
   حق بیمه خالص :  88,272,199.16  تومان
   کارمزد و هزینه اداری :  6,179,053.94  تومان
   حق بیمه نهایی :  94,451,253.1  تومان


# garanti annuity due
# مستمری ضمانتی

In [27]:
def garanti_annuity_due(Age=None  , period=None , pension_amount=None   , Warranty_period=None , add=False):
    global  data_G, Cost,Wage,EPV_G ,a,epv_G
    epv_G=[]
    data_G=pd.DataFrame()
    data_G['Age']=range(Age+Warranty_period,Age+1+Warranty_period)
    
    for j in range(0,period-Warranty_period):
        A_xn=[] 
        for age in range(Age+Warranty_period,Age+1+Warranty_period):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_G[j]=A_xn
  
        
    for N  in range(0,period-Warranty_period):
        epv_G.append( data_G[:][N])

    a=(1-Interest_Tabe.Nu_n[Warranty_period])/Interest_Tabe.d[Warranty_period]
    
    EPV= (((sum(epv_G).values)*((LifeTabel.l_x[Age+Warranty_period]/LifeTabel.l_x[Age])*(Interest_Tabe.Nu_n[Warranty_period]))))+a

    EPV=EPV*pension_amount
    Cost=(0.02*EPV[0])
    Wage=(0.05*EPV[0])
    
    EPV_G=EPV[0]+Cost+Wage
    if(add==False):  
        
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              مبلغ مستمری : ", f'{pension_amount:,}'     ,  "                   مدت ضمانت  :",Warranty_period)
        print("   حق بیمه خالص : "  ,  f'{round(EPV[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_G,2):,}' ," تومان")
        
        
    if(add==True):  
        return EPV_G

In [29]:
garanti_annuity_due(Age=25,period=20,pension_amount=5000000,Warranty_period=10, add=False)

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  5,000,000                    مدت ضمانت  : 10
   حق بیمه خالص :  35,860,087.96  تومان
   کارمزد و هزینه اداری :  2,510,206.16  تومان
   حق بیمه نهایی :  38,370,294.11  تومان


# Recommended product 1
# محصول پیشنهادی شماره 1

In [30]:
def product_1(Age=None   , pension_amount=None   ,  death_capital=0   , Capital_ratio=0  , add=False):
    period=20
    global  data_ , Cost,Wage,EPV_1 ,EPV_total,EPV_final
    death_capital=(pension_amount*Capital_ratio)+death_capital
    epv_=[]
    data_=pd.DataFrame()
    data_['Age']=range(Age,Age+1)
    
    for j in range(0,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_[j]=A_xn

        
    for N  in range(0,period):
        epv_.append( data_[:][N])
    EPV_1= (sum(epv_).values)*pension_amount
    

    
    
    global  EPV_TIA , Cost,Wage,data_TIA,EPV_2
    epv_TIA=[]
    data_TIA=pd.DataFrame()
    data_TIA['Age']=range(Age,Age+1)
    for j in range(0,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append (Interest_Tabe.Nu_n[j+1]*((LifeTabel.l_x[age+j]-LifeTabel.l_x[age+j+1])/LifeTabel.l_x[age]))
        data_TIA[j]=A_xn
    for N  in range(0,period):
        epv_TIA.append( data_TIA[:][N])
    EPV_2= (sum(epv_TIA).values)*death_capital
    EPV_total=EPV_2+EPV_1
    
    
    Cost=(0.02*EPV_total[0] )+(0.003*death_capital)
    Wage=(0.05*EPV_total[0])
    
    EPV_final=EPV_total+Cost+Wage
    if(add==False): 

        
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              مبلغ مستمری : ", f'{pension_amount:,}'     ,  "                   سرمایه فوت  :",death_capital)
        print("   حق بیمه خالص : "  ,  f'{round(EPV_total[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_final[0],2):,}' ," تومان")
        
        
    if(add==True): 
        return EPV_final[0]

In [31]:
product_1(Age=25,pension_amount=5000000,death_capital=1000000000,add=False)

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  5,000,000                    سرمایه فوت  : 1000000000
   حق بیمه خالص :  51,273,371.69  تومان
   کارمزد و هزینه اداری :  6,589,136.02  تومان
   حق بیمه نهایی :  57,862,507.71  تومان


# Recommended product 2
# محصول پیشنهادی شماره 2

In [32]:
def product_2(Age=None  , pension_amount=None  , death_capital=0   , Capital_ratio=0 ,add=True ):
    death_capital=(pension_amount*Capital_ratio)+death_capital
    Warranty_period=5
    period=20
    global  data_G, Cost,Wage,EPV_G ,a,epv_G
    epv_G=[]
    data_G=pd.DataFrame()
    data_G['Age']=range(Age+Warranty_period,Age+1+Warranty_period)
    
    for j in range(0,period-Warranty_period):
        A_xn=[] 
        for age in range(Age+Warranty_period,Age+1+Warranty_period):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_G[j]=A_xn

        
    for N  in range(0,period-Warranty_period):
        epv_G.append( data_G[:][N])

    a=(1-Interest_Tabe.Nu_n[Warranty_period])/Interest_Tabe.d[Warranty_period]
    
    EPV= (((sum(epv_G).values)*((LifeTabel.l_x[Age+Warranty_period]/LifeTabel.l_x[Age])*(Interest_Tabe.Nu_n[Warranty_period]))))+a
    EPV_1=EPV*pension_amount
    

    global  EPV_TIA , Cost,Wage,data_TIA,EPV_2
    epv_TIA=[]
    data_TIA=pd.DataFrame()
    data_TIA['Age']=range(Age,Age+1)
    for j in range(0,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append (Interest_Tabe.Nu_n[j+1]*((LifeTabel.l_x[age+j]-LifeTabel.l_x[age+j+1])/LifeTabel.l_x[age]))
        data_TIA[j]=A_xn
    for N  in range(0,period):
        epv_TIA.append( data_TIA[:][N])
    EPV_2= (sum(epv_TIA).values)*death_capital
    
    EPV_total=EPV_2+EPV_1
    
    
    
    Cost=(0.02*EPV_total[0])+(0.003*death_capital)
    Wage=(0.05*EPV_total[0])
    
    EPV_final_2=EPV_total[0]+Cost+Wage
    if(add==False): 
        
        print("سن : ",Age , "         مدت قرارداد : "  ,period  , "              مبلغ مستمری : ", f'{pension_amount:,}'     ,  "                   سرمایه فوت  :",f'{death_capital:,}')
        print("   حق بیمه خالص : "  ,  f'{round(EPV_total[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_final_2,2):,}' ," تومان")
        
    if(add==True): 
        return EPV_final_2 

In [33]:
product_2(Age=25,pension_amount=5000000,Capital_ratio=100,add=False)

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  5,000,000                    سرمایه فوت  : 500,000,000
   حق بیمه خالص :  42,248,530.75  تومان
   کارمزد و هزینه اداری :  4,457,397.15  تومان
   حق بیمه نهایی :  46,705,927.9  تومان


# Recommended product 3
# محصول پیشنهادی شماره 3

In [34]:
def  product_3(Age=None   , pension_amount=None , death_capital=0   , Capital_ratio=0 ,add=False):
    global  data_ , Cost,Wage,EPV_ , EPV_TIA , Cost,Wage,data_TIA,EPV_2
    death_capital=(pension_amount*Capital_ratio)+death_capital
    Rate_increase = 0.1
    epv_=[]
    data_=pd.DataFrame()
    data_['Age']=range(Age,Age+1)
    
    
    
    for j in range(0,14):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_[j]=A_xn

    for N  in range(0,14):
        epv_.append( data_[:][N])
    EPV_1= (sum(epv_).values)
    
    
    
    global  data_increasing , Cost,Wage,EPV_increasing
    epv_increasing=[]
    data_increasing=pd.DataFrame()
    data_increasing['Age']=range(Age+14,Age+15)
    
    for j in range(1,5):
        A_xn=[] 
        for age in range(Age+14,Age+15):
            A_xn.append (((1+Rate_increase)**j)*(Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_increasing[j]=A_xn
        data_increasing[j+1]=1
        
    for N  in range(1,5+1):
        epv_increasing.append( data_increasing[:][N])
    
    
    EPV_2= (sum(epv_increasing).values)
    EPV_3=(LifeTabel.l_x[Age+14]/LifeTabel.l_x[Age])*Interest_Tabe.Nu_n[14]
    
    
    EPV_total=(EPV_1+(EPV_2*EPV_3))*pension_amount
    
    

    epv_TIA=[]
    data_TIA=pd.DataFrame()
    data_TIA['Age']=range(Age,Age+1)
    for j in range(0,20):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append (Interest_Tabe.Nu_n[j+1]*((LifeTabel.l_x[age+j]-LifeTabel.l_x[age+j+1])/LifeTabel.l_x[age]))
        data_TIA[j]=A_xn
    for N  in range(0,20):
        epv_TIA.append( data_TIA[:][N])
        
    EPV_4= (sum(epv_TIA).values)*death_capital
    
    
    EPV_final_=EPV_total+EPV_4
    
    Cost=(0.02*EPV_final_[0])+(0.003*death_capital)
    Wage=(0.05*EPV_final_[0])
    
    EPV_final_3=EPV_final_[0]+Cost+Wage
    if(add==False): 
        
        print("سن : ",Age , "         مدت قرارداد : "  ,20  , "              مبلغ مستمری : ", f'{pension_amount:,}'     ,  "                   سرمایه فوت  :",f'{death_capital:,}')
        print("   حق بیمه خالص : "  ,  f'{round(EPV_final_[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی : "  ,  f'{round(EPV_final_3,2):,}' ," تومان")
        
    if(add==True):
        return EPV_final_3

In [36]:
product_3(Age=25,pension_amount=5000000, death_capital=750000000 ,add=False)

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  5,000,000                    سرمایه فوت  : 750,000,000
   حق بیمه خالص :  48,401,685.88  تومان
   کارمزد و هزینه اداری :  5,638,118.01  تومان
   حق بیمه نهایی :  54,039,803.89  تومان


# Special product
#   محصول ویژه

In [37]:
def annuity_due(Age=None  , period=None ):
    global  data_ ,EPV_
    epv_=[]
    data_=pd.DataFrame()
    data_['Age']=range(Age,Age+1)
    
    for j in range(0,period):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_[j]=A_xn

        
    for N  in range(0,period):
        epv_.append( data_[:][N])
    
    
    EPV= (sum(epv_).values)

    return EPV[0]

In [38]:
def Term_annuity_Monthly(Age=None , period=None  ):
    global  data_ ,EPV
    pension_amount=1
    epv_=[]
    data_=pd.DataFrame()
    Age=Age*12
    p=period*12
    data_['Age']=range(Age,Age+1)
    
    for j in range(0,p):
        A_xn=[] 
        for age in range(Age,Age+1):
            A_xn.append (          (InterestTabel_month.nu_n[j]) * (LifeTabel_month.L_x[j+age]/LifeTabel_month.L_x[age]) *(1/12) )
        data_[j]=A_xn

        
    for N  in range(0,p):
        epv_.append( data_[:][N])
    
    
    EPV= (sum(epv_).values)*pension_amount

    return round(EPV[0],5)

In [39]:
def  Special_product(Age=None   , pension_amount=None , death_capital=0   , Capital_ratio=0 , payment=None ):
    global  data_4 , Cost,Wage,EPV_ , EPV_TIA , Cost,Wage,data_TIA,EPV_2
    death_capital=(pension_amount*Capital_ratio)+death_capital
    Rate_increase = 0.1
    
    
    epv_=[]
    data_4=pd.DataFrame()
    data_4['Age']=range(Age+5,Age+1+5)
    for j in range(0,4):
        A_xn=[] 
        for age in range(Age+5,Age+1+5):
            A_xn.append ((Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_4[j]=A_xn

    for N  in range(0,4):
        epv_.append( data_4[:][N])
    EPV_1= (sum(epv_).values)*(LifeTabel.l_x[Age+5]/LifeTabel.l_x[Age])*Interest_Tabe.Nu_n[5]
    
    
    
    global  data_increasing , Cost,Wage,EPV_increasing
    epv_increasing=[]
    data_increasing=pd.DataFrame()
    data_increasing['Age']=range(Age+9,Age+10)
    
    for j in range(0,11):
        A_xn=[] 
        for age in range(Age+9,Age+10):
            A_xn.append (((1+Rate_increase)**j)*(Interest_Tabe.Nu_n[j])*((LifeTabel.l_x[j+age]/LifeTabel.l_x[age])))
        data_increasing[j]=A_xn

        
    for N  in range(0,11):
        epv_increasing.append( data_increasing[:][N])
    
    
    EPV_2= (sum(epv_increasing).values)*(LifeTabel.l_x[Age+9]/LifeTabel.l_x[Age])*Interest_Tabe.Nu_n[9]
    
    
    
    epv_TIA=[]
    data_TIA=pd.DataFrame()
    data_TIA['Age']=range(Age+5,Age+1+5)
    for j in range(0,15):
        A_xn=[] 
        for age in range(Age+5,Age+1+5):
            A_xn.append (Interest_Tabe.Nu_n[j+1]*((LifeTabel.l_x[age+j]-LifeTabel.l_x[age+j+1])/LifeTabel.l_x[age]))
        data_TIA[j]=A_xn
        
    for N  in range(0,15):
        epv_TIA.append( data_TIA[:][N])
        
    EPV_3= (sum(epv_TIA).values)*death_capital*(LifeTabel.l_x[Age+5]/LifeTabel.l_x[Age])*Interest_Tabe.Nu_n[5]
    
    
    
    
    EPV_total=((EPV_1+EPV_2)*pension_amount)+EPV_3
    
    
    Cost=(0.02*EPV_total[0])+(0.003*death_capital)
    Wage=(0.05*EPV_total[0])
    
    EPV_final_4=EPV_total[0]+Cost+Wage
    
    

    
    if(payment=='O'):     
        print("سن : ",Age , "         مدت قرارداد : "  ,20  , "              مبلغ مستمری : ", f'{pension_amount:,}'     ,  "                   سرمایه فوت  :",f'{death_capital:,}')
        print("   حق بیمه خالص یکجا : "  ,  f'{round(EPV_total[0],2):,}' ," تومان")
        print("   کارمزد و هزینه اداری : "  ,  f'{round(Cost+Wage,2):,}' ," تومان")
        print("   حق بیمه نهایی یکجا  : "  ,  f'{round(EPV_final_4,2):,}' ," تومان")
        print("\n")
        
        
    if(payment=='Y'):        
        #سالانه   

        Haghbeme_salane= EPV_total[0]/annuity_due(Age,5)
    
        list_y=[((.75*Haghbeme_salane*0.30) + (0.07*Haghbeme_salane)+0.002*death_capital)
              ,((0.75*Haghbeme_salane*0.175) + (0.07*Haghbeme_salane)+0.002*death_capital)
             ,((0.75*Haghbeme_salane*0.175) + (0.07*Haghbeme_salane)+0.002*death_capital)
             ,((0.75*Haghbeme_salane*0.175) + (0.07*Haghbeme_salane)+0.002*death_capital) 
             ,((0.75*Haghbeme_salane*0.175) + (0.07*Haghbeme_salane)+0.002*death_capital) ]
        print("سن : ",Age , "         مدت قرارداد : "  ,20  , "              مبلغ مستمری : ", f'{pension_amount:,}'     ,  "                   سرمایه فوت  :",f'{death_capital:,}')
        print("\n")
        for i in range(1,6):
               print( "سال  : ",i)
               print("   حق بیمه خالص سالانه : "  ,  f'{round(Haghbeme_salane,2):,}' ," تومان")
               print("   کارمزد و هزینه اداری : "  ,  f'{round(list_y[i-1],2):,}' ," تومان")
               print("   حق بیمه نهایی سالانه  : "  ,  f'{round(list_y[i-1]+Haghbeme_salane,2):,}' ," تومان")
               print("\n") 
             
            
            
    if(payment=='M'): 
        # ماهانه
            print("سن : ",Age , "         مدت قرارداد : "  ,20  , "              مبلغ مستمری : ", f'{pension_amount:,}'     ,  "                   سرمایه فوت  :",f'{death_capital:,}')
            print("\n")
            Haghbeme_Mah =EPV_total[0]/(Term_annuity_Monthly(Age,5)*12)
            
            list_m=[((.75*Haghbeme_Mah*0.30) + (0.07*Haghbeme_Mah)+0.002*death_capital)
                          ,((0.75*Haghbeme_Mah*0.175) + (0.07*Haghbeme_Mah)+0.002*death_capital)
                          ,((0.75*Haghbeme_Mah*0.175) + (0.07*Haghbeme_Mah)+0.002*death_capital)
                          ,((0.75*Haghbeme_Mah*0.175) + (0.07*Haghbeme_Mah)+0.002*death_capital) 
                          ,((0.75*Haghbeme_Mah*0.175) + (0.07*Haghbeme_Mah)+0.002*death_capital) ]
                                        
            for i in range(1,6):
                for j in range(1,13):
                 print("=========== ",'  سال'     ,i ,    "  ماه "   , j,"=========== ")
                 print("   حق بیمه خالص : "  ,  f'{round(Haghbeme_Mah,2):,}' ," تومان")
                 print("   کارمزد و هزینه اداری : "  ,  f'{round(list_m[i-1]/Term_annuity_Monthly(Age+i,1),2):,}' ," تومان")
                 print("   حق بیمه نهایی : "  ,  f'{round(Haghbeme_Mah+(list_m[i-1]/Term_annuity_Monthly(Age+i,1)),2):,}' ," تومان")  
                 print("\n")
        
        


In [40]:
Special_product(Age=25,pension_amount=10000000,Capital_ratio=100 , payment='O')

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  10,000,000                    سرمایه فوت  : 1,000,000,000
   حق بیمه خالص یکجا :  45,994,800.87  تومان
   کارمزد و هزینه اداری :  6,219,636.06  تومان
   حق بیمه نهایی یکجا  :  52,214,436.94  تومان




In [41]:
Special_product(Age=25,pension_amount=10000000,Capital_ratio=100 , payment='Y')

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  10,000,000                    سرمایه فوت  : 1,000,000,000


سال  :  1
   حق بیمه خالص سالانه :  11,955,981.48  تومان
   کارمزد و هزینه اداری :  5,527,014.54  تومان
   حق بیمه نهایی سالانه  :  17,482,996.01  تومان


سال  :  2
   حق بیمه خالص سالانه :  11,955,981.48  تومان
   کارمزد و هزینه اداری :  4,406,141.27  تومان
   حق بیمه نهایی سالانه  :  16,362,122.75  تومان


سال  :  3
   حق بیمه خالص سالانه :  11,955,981.48  تومان
   کارمزد و هزینه اداری :  4,406,141.27  تومان
   حق بیمه نهایی سالانه  :  16,362,122.75  تومان


سال  :  4
   حق بیمه خالص سالانه :  11,955,981.48  تومان
   کارمزد و هزینه اداری :  4,406,141.27  تومان
   حق بیمه نهایی سالانه  :  16,362,122.75  تومان


سال  :  5
   حق بیمه خالص سالانه :  11,955,981.48  تومان
   کارمزد و هزینه اداری :  4,406,141.27  تومان
   حق بیمه نهایی سالانه  :  16,362,122.75  تومان




In [42]:
Special_product(Age=25,pension_amount=10000000,Capital_ratio=100 , payment='M')

سن :  25          مدت قرارداد :  20               مبلغ مستمری :  10,000,000                    سرمایه فوت  : 1,000,000,000


===========    سال 1   ماه  1 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,450,698.1  تومان
   حق بیمه نهایی :  3,493,831.58  تومان


===========    سال 1   ماه  2 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,450,698.1  تومان
   حق بیمه نهایی :  3,493,831.58  تومان


===========    سال 1   ماه  3 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,450,698.1  تومان
   حق بیمه نهایی :  3,493,831.58  تومان


===========    سال 1   ماه  4 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,450,698.1  تومان
   حق بیمه نهایی :  3,493,831.58  تومان


===========    سال 1   ماه  5 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,450,698.1  تومان
   حق بیمه نهایی :  3,493,831.58  تومان


===========    سال 1   ما

   حق بیمه نهایی :  3,390,103.68  تومان


===========    سال 5   ماه  9 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,346,970.2  تومان
   حق بیمه نهایی :  3,390,103.68  تومان


===========    سال 5   ماه  10 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,346,970.2  تومان
   حق بیمه نهایی :  3,390,103.68  تومان


===========    سال 5   ماه  11 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,346,970.2  تومان
   حق بیمه نهایی :  3,390,103.68  تومان


===========    سال 5   ماه  12 =========== 
   حق بیمه خالص :  1,043,133.48  تومان
   کارمزد و هزینه اداری :  2,346,970.2  تومان
   حق بیمه نهایی :  3,390,103.68  تومان




# Available products

# Selected products

In [43]:
Selected_products= ( Defer_annuity_due(Age=25,period=15,pension_amount=5000000,Defer=5,add=True) + 
                        Term_insurance_annual(Age=25,period=20,suminsuraned=50000000,add=True) +
                   product_1(Age=25,pension_amount=500000,Capital_ratio=100,add=True)
                   
                   
                   )


In [44]:
print("   حق بیمه نهایی : "  ,  f'{round(Selected_products,2):,}' ," تومان")

   حق بیمه نهایی :  23,689,029.71  تومان
